In [15]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import matplotlib.pyplot as plt
import numpy as np
from dppy.finite_dpps import FiniteDPP

from dataloader.builder import build_dataset
from uncertainty_estimator.mcdue import MCDUEMasked
from uncertainty_estimator.masks import build_masks
from model.mlp import MLP
from experiments.utils.data import scale

In [17]:
config = {
    'dataset': 'concrete',
    'layers': [8, 128, 128, 64, 1]
}


In [18]:
dataset = build_dataset(config['dataset'])
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

x_train, x_val, _ = scale(x_train, x_val)
y_train, y_val, y_scaler = scale(y_train, y_val)


In [19]:
model = MLP(config['layers'])
model.fit((x_train, y_train), (x_val, y_val), dropout_rate=0.3)


Epoch [100/10000], Loss: 0.1652, Validation loss: 0.1549
Epoch [200/10000], Loss: 0.1415, Validation loss: 0.1457
Epoch [300/10000], Loss: 0.0875, Validation loss: 0.1431
Epoch [400/10000], Loss: 0.1036, Validation loss: 0.1521
Epoch [500/10000], Loss: 0.0857, Validation loss: 0.1129
Epoch [600/10000], Loss: 0.0735, Validation loss: 0.1259
Epoch [700/10000], Loss: 0.0569, Validation loss: 0.0874
Epoch [800/10000], Loss: 0.0762, Validation loss: 0.1260
Epoch [900/10000], Loss: 0.0810, Validation loss: 0.0869
Epoch [1000/10000], Loss: 0.0457, Validation loss: 0.0734
Epoch [1100/10000], Loss: 0.0766, Validation loss: 0.1615
Epoch [1200/10000], Loss: 0.0546, Validation loss: 0.1030
Epoch [1300/10000], Loss: 0.0490, Validation loss: 0.1009
Epoch [1400/10000], Loss: 0.0450, Validation loss: 0.0685
Epoch [1500/10000], Loss: 0.0604, Validation loss: 0.1157
Epoch [1600/10000], Loss: 0.0586, Validation loss: 0.1023
Epoch [1700/10000], Loss: 0.0515, Validation loss: 0.0786
Epoch [1800/10000], Los

In [20]:
mask = build_masks(['dpp'])['dpp']
estimator = MCDUEMasked(model, nn_runs=100, dropout_mask=mask)
uncertainty = estimator.estimate(x_val)

In [21]:
def rank(dpp):
    N = dpp.eig_vecs.shape[0]
    tol = np.max(dpp.L_eig_vals) * N * np.finfo(np.float).eps
    rank = np.count_nonzero(dpp.L_eig_vals > tol)
    return rank


In [42]:
k_matrix = mask.layer_correlations[1]
dpp = FiniteDPP('correlation', **{'K': np.abs(k_matrix)})
# dpp.sample_exact_k_dpp(10)

In [44]:
for i in range(4):
    dpp.sample_exact()
# print(rank(dpp))

In [45]:
for sample in dpp.list_of_samples:
    print(sample)


[ 0  1  8 54 12]
[32 36 53  8 63]
[38 12  8 62]
[56  8 43 31 20  9]
[21 15  5 23  8 52 36]
[10  0  8 34]
[17 30 44 48 56 45  8 55]
[53 16  8 47 37 63 54]


In [23]:
# compare rans for the original and positive correlation matrices
# ranks = []
# ranks_abs = []
# for i in range(2):
#     print(i)
#     model = MLP(config['layers'])
#     model.fit((x_train, y_train), (x_val, y_val), verbose=False, epochs=2000)
#     mask = build_masks(['dpp'])['dpp']
#     estimator = MCDUEMasked(model, nn_runs=100, dropout_mask=mask)
#     uncertainty = estimator.estimate(x_val)
#     k_matrix = mask.layer_correlations[0]
#     dpp = FiniteDPP('correlation', **{'K': k_matrix})
#     dpp.sample_exact_k_dpp(1)
#     ranks.append(rank(dpp))
#     dpp = FiniteDPP('correlation', **{'K': np.abs(k_matrix)})
#     dpp.sample_exact_k_dpp(1)
#     ranks_abs.append(rank(dpp))

In [24]:
# plt.scatter(ranks, ranks_abs)
# plt.plot((0, max(ranks)), (0, max(ranks)))
# plt.xlabel('full range ranks')
# plt.ylabel('abs ranks')